<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Generate-Data" data-toc-modified-id="Generate-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate Data</a></span></li><li><span><a href="#Standard-Convolution" data-toc-modified-id="Standard-Convolution-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Standard Convolution</a></span></li><li><span><a href="#Group-Convolution" data-toc-modified-id="Group-Convolution-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Group Convolution</a></span><ul class="toc-item"><li><span><a href="#NDArray-based" data-toc-modified-id="NDArray-based-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>NDArray-based</a></span></li><li><span><a href="#Symbol-based" data-toc-modified-id="Symbol-based-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Symbol-based</a></span></li></ul></li></ul></div>

# Generate Data

In [58]:
import numpy as np
from mxnet import nd

# Data format: [batch, channel, height, width]
# Weight format: [output_channels, in_channels, height, width]
num_filter   = 2
img_shape    = (1, 3, 2, 2)
weight_shape = (num_filter, img_shape[1], 1, 1)

data = nd.arange(img_shape[0]*img_shape[1]*img_shape[2]*img_shape[3]).reshape(img_shape)
weight = nd.arange(weight_shape[0] * weight_shape[1] * weight_shape[2] * weight_shape[3]).reshape(weight_shape)
bias = nd.zeros(num_filter)

print('Input:', data, '\n\nWeight:', weight, '\n\nBias:', bias)

Input: 
[[[[  0.   1.]
   [  2.   3.]]

  [[  4.   5.]
   [  6.   7.]]

  [[  8.   9.]
   [ 10.  11.]]]]
<NDArray 1x3x2x2 @cpu(0)> 

Weight: 
[[[[ 0.]]

  [[ 1.]]

  [[ 2.]]]


 [[[ 3.]]

  [[ 4.]]

  [[ 5.]]]]
<NDArray 2x3x1x1 @cpu(0)> 

Bias: 
[ 0.  0.]
<NDArray 2 @cpu(0)>


# Standard Convolution

In [59]:
conv = nd.Convolution(data, weight=weight, bias=bias, kernel=weight.shape[2:], num_filter=weight.shape[0])
print("Output of standard convolution:\n", conv)

Output of standard convolution:
 
[[[[ 20.  23.]
   [ 26.  29.]]

  [[ 56.  68.]
   [ 80.  92.]]]]
<NDArray 1x2x2x2 @cpu(0)>


# Group Convolution
## NDArray-based
注意: 首先需要调整 weight 的形状, 使满足 weight.shape[1] = 1, 从而 weight.shape[0] = in_channels * out_channels; 

num_group 应能被 in_channels 整除. 在本例中, data.shape = [1, 3, 2, 2], reshape 后的 weight.shape = [6, 1, 1, 1]. 若设置卷积中 num_group = 3, 则有 <br>
$$ data[0] * \{ weight[0], weight[1] \} → \{ out[0], out[1] \}$$ 
$$ data[1] * \{ weight[2], weight[3] \} → \{ out[2], out[3] \}$$ 
$$ data[2] * \{ weight[4], weight[5] \} → \{ out[4], out[5] \}$$  

In [60]:
in_channels  = data.shape[1]
out_channels = weight.shape[0]
num_kernels  = in_channels * out_channels
print('data   shape:', data.shape, '\nweight shape:', weight.shape)
 
weight0 = weight.reshape((-1, 1, 0, 0))
bias = nd.zeros(in_channels*out_channels)
print('\nshape of reshaped weight:', weight0.shape, '\nbias shape:', bias.shape)

conv = nd.Convolution(data, weight=weight0, bias=bias, kernel=weight.shape[2:],
                      num_filter=num_kernels,
                      num_group=3)
print("\nGroup Convolution Output:", conv)

data   shape: (1, 3, 2, 2) 
weight shape: (2, 3, 1, 1)

shape of reshaped weight: (6, 1, 1, 1) 
bias shape: (6,)

Group Convolution Output: 
[[[[  0.   0.]
   [  0.   0.]]

  [[  0.   1.]
   [  2.   3.]]

  [[  8.  10.]
   [ 12.  14.]]

  [[ 12.  15.]
   [ 18.  21.]]

  [[ 32.  36.]
   [ 40.  44.]]

  [[ 40.  45.]
   [ 50.  55.]]]]
<NDArray 1x6x2x2 @cpu(0)>


## Symbol-based
卷积中需要满足: 
$$ input\_channels = num\_group * weight\_channels $$
即
$$ num\_group = \frac{input.shape[1]}{weight.shape[1]} $$

In [62]:
import mxnet as mx

def get_symbol(num_group=1):
    data = mx.sym.Variable(name='data')
    conv = mx.sym.Convolution(data=data, kernel=(3, 3), stride=(1, 1), num_filter=24, num_group=num_group, name='conv')
    return conv

def infer_output_shape(sym, data_shape):
    all_layers = sym.get_internals()
    data_shape_dict = dict([('data', data_shape)])
    arg_shape, out_shape, aux_shape = sym.infer_shape(**data_shape_dict)
    arg_shape_dict = dict(zip(sym.list_arguments(), arg_shape))
    aux_shape_dict = dict(zip(sym.list_auxiliary_states(), aux_shape))

    # print the shape of parameters
    print("Shape of parameters:")
    for key in arg_shape_dict.keys():
        print('{}\t{}'.format(key, arg_shape_dict[key]))

    # infer the shape of feature maps
    print("\nShape of feature maps:")
    for name in all_layers.list_outputs():
        if name.find('output') != -1:  # 查找包含 "output" 的字符串
            arg_shape, out_shape, aux_shape = all_layers[name].infer_shape(**data_shape_dict)
            layer_shape_dict = dict(zip(all_layers[name].list_outputs(), out_shape))
            print(layer_shape_dict)

if __name__ == '__main__':
    data_shape = (1, 6, 256, 256)
    for num_group in [1, 2, 3, 6]:
        print("num-group =", num_group)
        sym = get_symbol(num_group)
        infer_output_shape(sym, data_shape)
        print("\n***********************************")

num-group = 1
Shape of parameters:
data	(1, 6, 256, 256)
conv_weight	(24, 6, 3, 3)
conv_bias	(24,)

Shape of feature maps:
{'conv_output': (1, 24, 254, 254)}

***********************************
num-group = 2
Shape of parameters:
data	(1, 6, 256, 256)
conv_weight	(24, 3, 3, 3)
conv_bias	(24,)

Shape of feature maps:
{'conv_output': (1, 24, 254, 254)}

***********************************
num-group = 3
Shape of parameters:
data	(1, 6, 256, 256)
conv_weight	(24, 2, 3, 3)
conv_bias	(24,)

Shape of feature maps:
{'conv_output': (1, 24, 254, 254)}

***********************************
num-group = 6
Shape of parameters:
data	(1, 6, 256, 256)
conv_weight	(24, 1, 3, 3)
conv_bias	(24,)

Shape of feature maps:
{'conv_output': (1, 24, 254, 254)}

***********************************
